# CUDA-Q Dynamics: Hamiltonian Simulation Tutorial

This notebook explores **Hamiltonian simulation with CUDA-Q** — a core primitive
for quantum chemistry, condensed-matter physics, and quantum machine learning.

We study three progressively richer systems:

| Part | System | Hamiltonian | Prediction |
|------|--------|-------------|------------|
| 1 | Single qubit in a field | H = ω/2 · Z | Larmor precession |
| 2 | Resonantly driven qubit | H = Ω/2 · X | Rabi oscillations |
| 3 | Two entangled qubits | H = J · Z⊗Z | Conditional phase evolution |
| 4 | Ising model | H = J·ZZ + h·X | Trotter approximation |

For each system we:
1. **Derive** the exact analytical solution
2. **Simulate** using `cudaq.evolve` (CUDA-Q dynamics backend)
3. **Implement** the equivalent unitary circuit and verify both agree

> **Prerequisites**: Sections 1–3 of the main CUDA-Q tutorial
> (quantum kernels, sampling, `cudaq.observe`)


In [ ]:
import cudaq
from cudaq import spin
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm

print(f"CUDA-Q version: {cudaq.__version__}")

# ── Pauli matrices (used throughout for analytics) ────────────────────────────
I2 = np.eye(2, dtype=complex)
X  = np.array([[0, 1 ], [1,  0]], dtype=complex)
Y  = np.array([[0, -1j], [1j, 0]], dtype=complex)
Z  = np.array([[1, 0 ], [0, -1]], dtype=complex)

def kron(*ops):
    """Tensor product of operators."""
    result = ops[0]
    for op in ops[1:]:
        result = np.kron(result, op)
    return result

def expect(psi, op):
    """Real part of ⟨ψ|op|ψ⟩."""
    return float(np.real(psi.conj() @ op @ psi))

def exact_evolve(H, psi0, t_vals, observables):
    """Exact time evolution using scipy matrix exponentiation."""
    results = {name: [] for name in observables}
    for t in t_vals:
        U   = expm(-1j * H * t)
        psi = U @ psi0
        for name, op in observables.items():
            results[name].append(expect(psi, op))
    return {k: np.array(v) for k, v in results.items()}

# ── Check if CUDA-Q dynamics target is available ──────────────────────────────
try:
    cudaq.set_target("dynamics")
    HAS_DYNAMICS = True
    print("cudaq.evolve (dynamics target) ✓")
except Exception as e:
    HAS_DYNAMICS = False
    print(f"Dynamics target not available: {e}")
    print("-> All dynamics cells will fall back to scipy.linalg.expm (exact).")

# Reset to default gate-model target for circuit sections
cudaq.set_target("qpp-cpu")


## Background: Time Evolution & Hamiltonian Dynamics

The Schrödinger equation governs how a quantum state evolves:

$$i\hbar \frac{d}{dt}|\psi(t)\rangle = H|\psi(t)\rangle$$

For a **time-independent** Hamiltonian, the solution is the unitary:

$$|\psi(t)\rangle = e^{-iHt/\hbar}|\psi_0\rangle \quad (\text{setting } \hbar = 1)$$

### Why It Matters for Quantum Computing

Simulating this time evolution is a core quantum-computing primitive:
- **Quantum chemistry**: molecular ground states & reaction pathways
- **Condensed matter**: phase transitions, magnetism
- **Quantum machine learning**: encoding physical inductive biases

### Two Ways to Simulate in CUDA-Q

| Approach | API | When to Use |
|----------|-----|-------------|
| **Dynamics backend** | `cudaq.evolve()` | Continuous-time ODE solver (Schrödinger / Lindblad) |
| **Gate circuit** | `@cudaq.kernel` | Discrete Trotter steps or exact 1- / 2-qubit gates |

We use **both** and compare them throughout this notebook.

### The Circuit–Dynamics Connection

Every unitary $U = e^{-iHt}$ can (in principle) be compiled to a quantum circuit.
For common Pauli Hamiltonians, the mapping is exact:

| Hamiltonian | Unitary | Circuit gate |
|-------------|---------|--------------|
| $\frac{\omega}{2} Z$ | $e^{-i\omega t/2\, Z}$ | `rz(ω·t)` |
| $\frac{\Omega}{2} X$ | $e^{-i\Omega t/2\, X}$ | `rx(Ω·t)` |
| $J\, Z\otimes Z$ | $e^{-iJt\, ZZ}$ | CNOT · `rz(2Jt)` · CNOT |


## Part 1: Larmor Precession — H = ω/2 · Z

**Physical picture**: A spin-½ particle in a static magnetic field
$\mathbf{B} = B_0\hat{z}$ — the qubit analogue of an NMR spin.

### Analytical Solution

Starting from $|{+}\rangle = (|0\rangle + |1\rangle)/\sqrt{2}$
(Bloch vector pointing along +X):

$$|\psi(t)\rangle = e^{-i\omega t/2\, Z}|{+}\rangle
= \frac{e^{-i\omega t/2}|0\rangle + e^{+i\omega t/2}|1\rangle}{\sqrt{2}}$$

Expectation values:

$$\langle X(t)\rangle = \cos(\omega t)$$
$$\langle Y(t)\rangle = \sin(\omega t)$$
$$\langle Z(t)\rangle = 0$$

The Bloch vector **precesses in the XY plane** with angular frequency ω.

### Equivalent Circuit

$$e^{-i\omega t/2\, Z} = \text{Rz}(\omega t)$$

```
q[0]: ── H ── Rz(ωt) ──
```


In [ ]:
# ─── Analytical + Bloch-sphere trajectory ─────────────────────────────────────
omega = 2.0  # rad/time-unit
T     = 2 * np.pi / omega  # one full precession
t     = np.linspace(0, 2 * T, 400)

# Analytical formulas (derived above)
x_an = np.cos(omega * t)
y_an = np.sin(omega * t)
z_an = np.zeros_like(t)

# Verify with exact matrix exponentiation
H_larmor = 0.5 * omega * Z
psi0_plus = np.array([1, 1], dtype=complex) / np.sqrt(2)

num = exact_evolve(H_larmor, psi0_plus, t,
                   {"X": X, "Y": Y, "Z": Z})

fig, axes = plt.subplots(1, 3, figsize=(13, 4))

for ax, (comp, an_vals, num_vals, color) in zip(axes, [
    ("⟨X⟩", x_an, num["X"], "steelblue"),
    ("⟨Y⟩", y_an, num["Y"], "coral"),
    ("⟨Z⟩", z_an, num["Z"], "seagreen"),
]):
    ax.plot(t, an_vals, "-",  color=color,   lw=2.5, label="Analytical")
    ax.plot(t, num_vals,"--", color="black", lw=1.2, alpha=0.6, label="Matrix exp")
    ax.set_xlabel("Time  t");  ax.set_ylabel(comp)
    ax.set_title(f"Larmor Precession: {comp}")
    ax.legend(fontsize=9);  ax.grid(True, alpha=0.3)

plt.suptitle(r"H = ω/2 · Z,   ω = " + f"{omega},   |ψ₀⟩ = |+⟩", fontsize=12)
plt.tight_layout()
plt.show()
print("Analytical and matrix-exponentiation results match ✓")


In [ ]:
# ─── CUDA-Q Dynamics vs. Circuit ─────────────────────────────────────────────
from cudaq import spin

omega = 2.0
T     = 2 * np.pi / omega
t_vals = np.linspace(0, 2 * T, 80)

# ── (A) CUDA-Q cudaq.evolve ───────────────────────────────────────────────────
dyn_x, dyn_y, dyn_z = None, None, None

if HAS_DYNAMICS:
    try:
        cudaq.set_target("dynamics")
        H_cudaq = 0.5 * omega * spin.z(0)
        dimensions = {0: 2}

        from cudaq import ScipyZvodeIntegrator
        # Initial state |+⟩ as density matrix ρ = |+⟩⟨+|
        # (use cupy arrays instead of numpy for GPU-backed dynamics)
        rho0 = cudaq.State.from_data(
            np.array([[0.5, 0.5], [0.5, 0.5]], dtype=complex))

        schedule = cudaq.Schedule(t_vals.tolist(), ["time"])

        result = cudaq.evolve(
            H_cudaq, dimensions, schedule, rho0,
            observables=[spin.x(0), spin.y(0), spin.z(0)],
            collapse_operators=[],
            store_intermediate_results=cudaq.IntermediateResultSave.EXPECTATION_VALUE,
            integrator=ScipyZvodeIntegrator())

        ev    = result.expectation_values()
        # ev[step][obs_idx].expectation() -> float
        dyn_x = np.array([ev[i][0].expectation() for i in range(len(ev))])
        dyn_y = np.array([ev[i][1].expectation() for i in range(len(ev))])
        dyn_z = np.array([ev[i][2].expectation() for i in range(len(ev))])
        print("cudaq.evolve: success ✓")
        cudaq.set_target("qpp-cpu")
    except Exception as err:
        print(f"cudaq.evolve failed: {err} — using scipy fallback")
        cudaq.set_target("qpp-cpu")

# Scipy fallback if dynamics unavailable
if dyn_x is None:
    H_larmor = 0.5 * omega * Z
    psi0_np  = np.array([1/np.sqrt(2), 1/np.sqrt(2)], dtype=complex)
    num = exact_evolve(H_larmor, psi0_np, t_vals, {"X": X, "Y": Y, "Z": Z})
    dyn_x, dyn_y, dyn_z = num["X"], num["Y"], num["Z"]

# ── (B) Circuit: H · Rz(ωt) ──────────────────────────────────────────────────
@cudaq.kernel
def larmor_circuit(omega: float, t: float):
    q = cudaq.qvector(1)
    h(q[0])           # |+⟩
    rz(omega * t, q[0])  # = exp(-iωt/2 · Z)

print("\nLarmor circuit (ωt = π/2):")
print(cudaq.draw(larmor_circuit, omega, np.pi / (2*omega)))

t_circ  = np.linspace(0, 2*T, 40)   # fewer points — one kernel call per point
circ_x  = [cudaq.observe(larmor_circuit, spin.x(0), omega, float(ti)).expectation()
           for ti in t_circ]
circ_y  = [cudaq.observe(larmor_circuit, spin.y(0), omega, float(ti)).expectation()
           for ti in t_circ]
circ_z  = [cudaq.observe(larmor_circuit, spin.z(0), omega, float(ti)).expectation()
           for ti in t_circ]

# ── Analytical reference ──────────────────────────────────────────────────────
t_ref = np.linspace(0, 2*T, 400)
x_an  = np.cos(omega * t_ref)
y_an  = np.sin(omega * t_ref)

# ── Plot ──────────────────────────────────────────────────────────────────────
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

# Time traces
for comp, an, dyn, circ in [
    ("⟨X⟩", x_an, dyn_x, circ_x),
    ("⟨Y⟩", y_an, dyn_y, circ_y),
]:
    label_dyn  = "cudaq.evolve" if HAS_DYNAMICS else "scipy exact"
    ax1.plot(t_ref,  an,   "-",  lw=2,   alpha=0.8, label=f"Analytical {comp}")
    ax1.plot(t_vals, dyn,  "o",  ms=4,   alpha=0.7, label=f"Dynamics {comp}")
    ax1.plot(t_circ, circ, "^",  ms=5,   alpha=0.7, label=f"Circuit {comp}")

ax1.set_xlabel("Time  t"); ax1.set_ylabel("Expectation value")
ax1.set_title("Larmor Precession: Three Approaches")
ax1.legend(fontsize=7.5, ncol=2); ax1.grid(True, alpha=0.3)

# Bloch sphere trajectory (top view: XY plane)
theta = np.linspace(0, 2*np.pi, 200)
ax2.plot(np.cos(theta), np.sin(theta), "lightgray", lw=1)
ax2.plot(np.cos(omega*t_vals), np.sin(omega*t_vals), "b-", lw=2, label="Analytic trajectory")
ax2.scatter(dyn_x, dyn_y, s=18, color="orange", zorder=5, label="Dynamics points")
ax2.scatter([1], [0], s=120, color="green",  zorder=6, label="Start |+⟩")
ax2.set_aspect("equal"); ax2.set_xlim(-1.25, 1.25); ax2.set_ylim(-1.25, 1.25)
ax2.set_xlabel("⟨X⟩"); ax2.set_ylabel("⟨Y⟩")
ax2.set_title("Bloch Sphere (XY plane): precession circle")
ax2.legend(fontsize=8); ax2.grid(True, alpha=0.3)

plt.suptitle("H = ω/2 · Z — Larmor Precession", fontsize=12)
plt.tight_layout(); plt.show()


## Part 2: Rabi Oscillations — H = Ω/2 · X

**Physical picture**: A resonant microwave pulse drives transitions between
$|0\rangle$ and $|1\rangle$ — the workhorse of qubit control.

### Analytical Solution

Starting from $|0\rangle$ (Bloch vector at +Z):

$$|\psi(t)\rangle = e^{-i\Omega t/2\, X}|0\rangle
= \cos\!\left(\frac{\Omega t}{2}\right)|0\rangle
  - i\sin\!\left(\frac{\Omega t}{2}\right)|1\rangle$$

Expectation values:

$$\langle X(t)\rangle = 0$$
$$\langle Y(t)\rangle = -\sin(\Omega t)$$
$$\langle Z(t)\rangle = \cos(\Omega t)$$

The Bloch vector **oscillates between +Z and −Z** (between $|0\rangle$ and $|1\rangle$)
at the **Rabi frequency** Ω.
At $t = \pi/\Omega$: perfect $|0\rangle \to |1\rangle$ population inversion!

### Equivalent Circuit

$$e^{-i\Omega t/2\, X} = \text{Rx}(\Omega t)$$

```
q[0]: ── Rx(Ωt) ──
```


In [ ]:
# ─── Rabi Oscillations: Analytical + Dynamics + Circuit ───────────────────────
Omega = 1.5  # Rabi frequency
T     = 2 * np.pi / Omega
t_ref = np.linspace(0, 2*T, 400)

# ── Analytical ────────────────────────────────────────────────────────────────
x_an = np.zeros_like(t_ref)
y_an = -np.sin(Omega * t_ref)
z_an =  np.cos(Omega * t_ref)

# Verify with expm
H_rabi = 0.5 * Omega * X
psi0_0 = np.array([1, 0], dtype=complex)
num = exact_evolve(H_rabi, psi0_0, t_ref, {"X": X, "Y": Y, "Z": Z})

# ── CUDA-Q Dynamics ───────────────────────────────────────────────────────────
t_vals  = np.linspace(0, 2*T, 80)
dyn_x, dyn_y, dyn_z = None, None, None

if HAS_DYNAMICS:
    try:
        cudaq.set_target("dynamics")
        from cudaq import ScipyZvodeIntegrator
        H_cudaq = 0.5 * Omega * spin.x(0)
        dimensions = {0: 2}
        # |0⟩ density matrix ρ = |0⟩⟨0|
        rho0 = cudaq.State.from_data(
            np.array([[1.0, 0.0], [0.0, 0.0]], dtype=complex))
        schedule = cudaq.Schedule(t_vals.tolist(), ["time"])

        result = cudaq.evolve(
            H_cudaq, dimensions, schedule, rho0,
            observables=[spin.x(0), spin.y(0), spin.z(0)],
            collapse_operators=[],
            store_intermediate_results=cudaq.IntermediateResultSave.EXPECTATION_VALUE,
            integrator=ScipyZvodeIntegrator())

        ev    = result.expectation_values()
        dyn_x = np.array([ev[i][0].expectation() for i in range(len(ev))])
        dyn_y = np.array([ev[i][1].expectation() for i in range(len(ev))])
        dyn_z = np.array([ev[i][2].expectation() for i in range(len(ev))])
        cudaq.set_target("qpp-cpu")
        print("cudaq.evolve (Rabi): success ✓")
    except Exception as err:
        print(f"cudaq.evolve: {err}  — scipy fallback")
        cudaq.set_target("qpp-cpu")

if dyn_x is None:
    num_d = exact_evolve(H_rabi, psi0_0, t_vals, {"X": X, "Y": Y, "Z": Z})
    dyn_x, dyn_y, dyn_z = num_d["X"], num_d["Y"], num_d["Z"]

# ── Circuit: Rx(Ωt) ───────────────────────────────────────────────────────────
@cudaq.kernel
def rabi_circuit(Omega: float, t: float):
    q = cudaq.qvector(1)
    rx(Omega * t, q[0])   # |0⟩ initial, Rx = exp(-iΩt/2 · X)

print("\nRabi circuit (Ωt = π  →  π-pulse):")
print(cudaq.draw(rabi_circuit, Omega, np.pi / Omega))

t_circ = np.linspace(0, 2*T, 40)
circ_y = [cudaq.observe(rabi_circuit, spin.y(0), Omega, float(ti)).expectation()
          for ti in t_circ]
circ_z = [cudaq.observe(rabi_circuit, spin.z(0), Omega, float(ti)).expectation()
          for ti in t_circ]

# ── Plot ──────────────────────────────────────────────────────────────────────
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

# ⟨Z⟩ (population inversion)
label_dyn = "cudaq.evolve" if HAS_DYNAMICS else "scipy exact"
ax1.plot(t_ref,  z_an,  "-",  lw=2.5,  color="steelblue", label="Analytical ⟨Z⟩")
ax1.plot(t_vals, dyn_z, "o",  ms=4,    color="coral",     label=f"{label_dyn} ⟨Z⟩")
ax1.plot(t_circ, circ_z,"^",  ms=5,    color="seagreen",  label="Circuit ⟨Z⟩")
ax1.axhline(-1, color="gray", ls=":", lw=1, label="|1⟩")
ax1.axhline(+1, color="gray", ls=":", lw=1, label="|0⟩")
ax1.axvline(np.pi/Omega, color="purple", ls="--", lw=1, label="π-pulse")
ax1.set_xlabel("Time  t"); ax1.set_ylabel("⟨Z⟩")
ax1.set_title("Rabi Oscillations: Population Inversion")
ax1.legend(fontsize=8); ax1.grid(True, alpha=0.3)

# Bloch sphere side view (YZ plane)
theta = np.linspace(0, 2*np.pi, 200)
ax2.plot(np.sin(theta), np.cos(theta), "lightgray", lw=1)  # circle outline
ax2.plot(-np.sin(Omega*t_ref), np.cos(Omega*t_ref),
         "b-", lw=2, alpha=0.8, label="Analytic trajectory")
ax2.scatter(dyn_y, dyn_z, s=18, color="orange", zorder=5, label="Dynamics")
ax2.scatter([0], [1], s=120, color="green", zorder=6, label="Start |0⟩")
ax2.scatter([0], [-1], s=120, color="red", marker="*", zorder=6, label="|1⟩")
ax2.set_aspect("equal"); ax2.set_xlim(-1.3, 1.3); ax2.set_ylim(-1.3, 1.3)
ax2.set_xlabel("⟨Y⟩"); ax2.set_ylabel("⟨Z⟩")
ax2.set_title("Bloch Sphere (YZ plane): Rabi oscillation")
ax2.legend(fontsize=8); ax2.grid(True, alpha=0.3)

plt.suptitle(r"H = Ω/2 · X  —  Rabi Oscillations,  Ω = " + f"{Omega}", fontsize=12)
plt.tight_layout(); plt.show()

# Key time points
print(f"\n{'Time':>12}  {'⟨Z⟩ analytic':>14}  {'Description'}")
print("-" * 48)
for frac, desc in [(0,"start |0⟩"), (0.5,"π/2-pulse"), (1,"π-pulse |1⟩"),
                    (1.5,"3π/2-pulse"), (2,"2π full rotation |0⟩")]:
    tp = frac * np.pi / Omega
    zv = np.cos(Omega * tp)
    print(f"t = {frac:.1f}π/Ω  {zv:>14.3f}  {desc}")


## Part 3: Two-Qubit ZZ Coupling — H = J · Z⊗Z

**Physical picture**: Two coupled spin-½ particles with Ising-type interaction —
appears in superconducting transmon qubits, trapped ions, and NMR.

### Analytical Solution

Starting from $|{+}\rangle_0 \otimes |0\rangle_1$
(qubit 0 in superposition, qubit 1 as spectator in +Z eigenstate):

$$|\psi(t)\rangle = e^{-iJt\, Z_0 Z_1}|{+}0\rangle
= \frac{e^{-iJt}|00\rangle + e^{+iJt}|10\rangle}{\sqrt{2}}$$

Expectation values (qubit 0):

$$\langle X_0(t)\rangle = \cos(2Jt)$$
$$\langle Y_0(t)\rangle = \sin(2Jt)$$
$$\langle Z_0(t)\rangle = 0$$

Qubit 1 stays fixed: $\langle Z_1(t)\rangle = 1$

**Key insight**: Qubit 1 is in a Z-eigenstate, so it acts as a classical
control — it kicks a conditional phase onto qubit 0's superposition.
This is exactly the mechanism behind a **CZ gate** at $t = \pi/(2J)$.

### Exact Circuit Decomposition

$$e^{-iJt\, Z_0 Z_1} = \text{CNOT} \cdot \bigl[I\otimes \text{Rz}(2Jt)\bigr] \cdot \text{CNOT}$$

```
q[0]: ── H ──●──────────●──
             │          │
q[1]: ───────⊕── Rz(2Jt) ──⊕──
```


In [ ]:
# ─── ZZ Coupling: Analytical + Dynamics + Circuit ─────────────────────────────
J    = 1.2  # coupling strength
T    = np.pi / J   # half-period of ⟨X₀⟩ oscillation
t_ref = np.linspace(0, 2*T, 400)

# ── Analytical ────────────────────────────────────────────────────────────────
x0_an = np.cos(2 * J * t_ref)
y0_an = np.sin(2 * J * t_ref)
z0_an = np.zeros_like(t_ref)
z1_an = np.ones_like(t_ref)

# Verify with expm on 4-dim space
H_zz  = J * kron(Z, Z)
psi0_plus0 = np.array([1/np.sqrt(2), 0, 1/np.sqrt(2), 0], dtype=complex)
# |+0⟩ = (|00⟩ + |10⟩)/√2  in basis {|00⟩,|01⟩,|10⟩,|11⟩}

X0 = kron(X, I2)
Y0 = kron(Y, I2)
Z0 = kron(Z, I2)
Z1 = kron(I2, Z)

num = exact_evolve(H_zz, psi0_plus0, t_ref, {"X0": X0, "Y0": Y0, "Z0": Z0, "Z1": Z1})

# ── CUDA-Q Dynamics ───────────────────────────────────────────────────────────
t_vals = np.linspace(0, 2*T, 80)
dyn_x0, dyn_y0, dyn_z0, dyn_z1 = None, None, None, None

if HAS_DYNAMICS:
    try:
        cudaq.set_target("dynamics")
        from cudaq import ScipyZvodeIntegrator
        H_cudaq    = J * spin.z(0) * spin.z(1)
        dimensions = {0: 2, 1: 2}
        # |+0⟩ as 4×4 density matrix  ρ = |+0⟩⟨+0|
        psi0_vec = np.array([1/np.sqrt(2), 0, 1/np.sqrt(2), 0], dtype=complex)
        rho0 = cudaq.State.from_data(np.outer(psi0_vec, psi0_vec.conj()))
        schedule = cudaq.Schedule(t_vals.tolist(), ["time"])

        result = cudaq.evolve(
            H_cudaq, dimensions, schedule, rho0,
            observables=[spin.x(0), spin.y(0), spin.z(0), spin.z(1)],
            collapse_operators=[],
            store_intermediate_results=cudaq.IntermediateResultSave.EXPECTATION_VALUE,
            integrator=ScipyZvodeIntegrator())

        ev     = result.expectation_values()
        dyn_x0 = np.array([ev[i][0].expectation() for i in range(len(ev))])
        dyn_y0 = np.array([ev[i][1].expectation() for i in range(len(ev))])
        dyn_z0 = np.array([ev[i][2].expectation() for i in range(len(ev))])
        dyn_z1 = np.array([ev[i][3].expectation() for i in range(len(ev))])
        cudaq.set_target("qpp-cpu")
        print("cudaq.evolve (ZZ): success ✓")
    except Exception as err:
        print(f"cudaq.evolve: {err}  — scipy fallback")
        cudaq.set_target("qpp-cpu")

if dyn_x0 is None:
    nd = exact_evolve(H_zz, psi0_plus0, t_vals,
                      {"X0": X0, "Y0": Y0, "Z0": Z0, "Z1": Z1})
    dyn_x0, dyn_y0, dyn_z0, dyn_z1 = nd["X0"], nd["Y0"], nd["Z0"], nd["Z1"]

# ── Circuit: H · CNOT · Rz(2Jt) · CNOT ──────────────────────────────────────
@cudaq.kernel
def zz_circuit(J: float, t: float):
    q = cudaq.qvector(2)
    h(q[0])                  # qubit 0: |+⟩
    # qubit 1 stays |0⟩ — Z eigenstate, acts as spectator
    cx(q[0], q[1])
    rz(2 * J * t, q[1])      # conditional Rz via ZZ decomposition
    cx(q[0], q[1])

print("\nZZ circuit (Jt = π/4  →  CZ gate moment):")
print(cudaq.draw(zz_circuit, J, np.pi / (4*J)))

t_circ  = np.linspace(0, 2*T, 40)
circ_x0 = [cudaq.observe(zz_circuit, spin.x(0), J, float(ti)).expectation()
            for ti in t_circ]
circ_y0 = [cudaq.observe(zz_circuit, spin.y(0), J, float(ti)).expectation()
            for ti in t_circ]

# ── Plot ──────────────────────────────────────────────────────────────────────
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

label_dyn = "cudaq.evolve" if HAS_DYNAMICS else "scipy exact"

# ⟨X₀⟩
axes[0].plot(t_ref, x0_an,   "-",  lw=2.5, color="steelblue", label="Analytical")
axes[0].plot(t_vals, dyn_x0, "o",  ms=4,   color="coral",     label=label_dyn)
axes[0].plot(t_circ, circ_x0,"^",  ms=5,   color="seagreen",  label="Circuit")
axes[0].axvline(np.pi/(4*J), color="purple", ls="--", lw=1.5, label="CZ gate (Jt=π/4)")
axes[0].set_xlabel("t"); axes[0].set_ylabel("⟨X₀(t)⟩")
axes[0].set_title("Qubit 0 — X expectation")
axes[0].legend(fontsize=8); axes[0].grid(True, alpha=0.3)

# ⟨Y₀⟩
axes[1].plot(t_ref, y0_an,   "-",  lw=2.5, color="steelblue", label="Analytical")
axes[1].plot(t_vals, dyn_y0, "o",  ms=4,   color="coral",     label=label_dyn)
axes[1].plot(t_circ, circ_y0,"^",  ms=5,   color="seagreen",  label="Circuit")
axes[1].set_xlabel("t"); axes[1].set_ylabel("⟨Y₀(t)⟩")
axes[1].set_title("Qubit 0 — Y expectation")
axes[1].legend(fontsize=8); axes[1].grid(True, alpha=0.3)

# Bloch sphere XY view
theta = np.linspace(0, 2*np.pi, 200)
axes[2].plot(np.cos(theta), np.sin(theta), "lightgray", lw=1)
axes[2].plot(np.cos(2*J*t_ref), np.sin(2*J*t_ref),
             "b-", lw=2, label="Analytic circle")
axes[2].scatter(dyn_x0, dyn_y0, s=18, color="orange", zorder=5, label="Dynamics")
axes[2].scatter([1], [0], s=120, color="green", zorder=6, label="Start |+⟩")
axes[2].set_aspect("equal"); axes[2].set_xlim(-1.3, 1.3); axes[2].set_ylim(-1.3, 1.3)
axes[2].set_xlabel("⟨X₀⟩"); axes[2].set_ylabel("⟨Y₀⟩")
axes[2].set_title("Qubit 0 Bloch sphere (XY plane)")
axes[2].legend(fontsize=8); axes[2].grid(True, alpha=0.3)

plt.suptitle(f"H = J·Z⊗Z   J = {J},   initial state |+0⟩", fontsize=12)
plt.tight_layout(); plt.show()

print(f"\nSpecial time: t = π/(2J) = {np.pi/(2*J):.3f}")
print(f"  At this time the ZZ evolution = CZ gate (up to local phases)")


## Part 4: Trotterization — H = J·Z⊗Z + h·X₀

When the Hamiltonian has **non-commuting terms**, no single gate implements
the exact evolution. We use the **Trotter–Suzuki decomposition**:

### First-Order Trotter

$$e^{-iH\Delta t} \approx e^{-iJ\Delta t\, ZZ} \cdot e^{-ih\Delta t\, X_0}
\quad +\; O(\Delta t^2)$$

Per Trotter step, this becomes two gates:

```
Step k:
  q[0]: ──●────────────●── Rx(2h·Δt) ──
          │            │
  q[1]: ──⊕── Rz(2J·Δt) ──⊕───────────
```

Repeat this block **n = T/Δt** times to simulate up to time T.

### Trotter Error

The error scales as $O(\Delta t)$ per step → $O(T\,\Delta t)$ total.
Smaller Δt (more steps) = better accuracy but more circuit depth.

### Second-Order Trotter (Suzuki)

$$e^{-iH\Delta t} \approx
e^{-iH_1\Delta t/2} \cdot e^{-iH_2\Delta t} \cdot e^{-iH_1\Delta t/2}
\quad +\; O(\Delta t^3)$$


In [ ]:
# ─── Trotterization: Circuit vs Exact ────────────────────────────────────────
J = 1.0   # ZZ coupling
h = 0.8   # transverse field strength
T = 3.0   # total simulation time

# ── Exact reference (scipy expm) ──────────────────────────────────────────────
H_exact = J * kron(Z, Z) + h * kron(X, I2)
psi0_np = np.array([1/np.sqrt(2), 0, 1/np.sqrt(2), 0], dtype=complex)  # |+0⟩

t_ref  = np.linspace(0, T, 300)
ref_x0 = np.array([
    expect(expm(-1j * H_exact * t) @ psi0_np, kron(X, I2))
    for t in t_ref])
ref_z0 = np.array([
    expect(expm(-1j * H_exact * t) @ psi0_np, kron(Z, I2))
    for t in t_ref])

# ── Trotter Circuit ───────────────────────────────────────────────────────────
# One Trotter step: exp(-iJ*dt*ZZ) · exp(-ih*dt*X)
@cudaq.kernel
def trotter_step(dt: float, J: float, h: float):
    """Single first-order Trotter step for H = J·ZZ + h·X₀"""
    q = cudaq.qvector(2)
    # Term 1: exp(-iJdt ZZ) = CNOT · Rz(2J·dt) · CNOT
    cx(q[0], q[1])
    rz(2 * J * dt, q[1])
    cx(q[0], q[1])
    # Term 2: exp(-ihdt X₀) = Rx(2h·dt)
    rx(2 * h * dt, q[0])

# N-step circuit by repeating the kernel N times
@cudaq.kernel
def trotter_n_steps(n_steps: int, dt: float, J: float, h: float):
    q = cudaq.qvector(2)
    h(q[0])   # initial |+0⟩

    # Trotter loop: n_steps repetitions
    for _ in range(n_steps):
        # exp(-iJ·dt·ZZ)
        cx(q[0], q[1])
        rz(2 * J * dt, q[1])
        cx(q[0], q[1])
        # exp(-ih·dt·X₀)
        rx(2 * h * dt, q[0])

print("One Trotter step circuit:")
print(cudaq.draw(trotter_step, T/10, J, h))

# ── Convergence study ─────────────────────────────────────────────────────────
print("Trotter convergence study...")
n_steps_list = [5, 10, 20, 40]
colors = ["#e74c3c", "#e67e22", "#2ecc71", "#2980b9"]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(t_ref, ref_x0, "k-", lw=2.5, label="Exact (scipy expm)")
ax2.plot(t_ref, ref_z0, "k-", lw=2.5, label="Exact (scipy expm)")

errors = []
for n, col in zip(n_steps_list, colors):
    dt = T / n
    # Measure at each Trotter step's endpoint
    t_trott = np.linspace(0, T, n+1)
    x0_vals, z0_vals = [], []

    for k in range(n+1):
        if k == 0:
            x0_vals.append(0.0)   # ⟨X⟩ of |+0⟩ is actually 1… see note below
            # Note: |+0⟩ has ⟨X₀⟩=1, ⟨Z₀⟩=0 — we approximate with 0 steps
            x0_vals[-1] = 1.0
            z0_vals.append(0.0)
        else:
            x_e = cudaq.observe(trotter_n_steps, spin.x(0), k, dt, J, h).expectation()
            z_e = cudaq.observe(trotter_n_steps, spin.z(0), k, dt, J, h).expectation()
            x0_vals.append(x_e)
            z0_vals.append(z_e)

    ax1.plot(t_trott, x0_vals, "o-", ms=4, color=col, lw=1.5,
             label=f"Trotter n={n}  (Δt={dt:.2f})", alpha=0.85)
    ax2.plot(t_trott, z0_vals, "o-", ms=4, color=col, lw=1.5,
             label=f"Trotter n={n}  (Δt={dt:.2f})", alpha=0.85)

    # Total error at final time
    x_final_exact  = ref_x0[-1]
    x_final_trott  = x0_vals[-1]
    errors.append(abs(x_final_exact - x_final_trott))

ax1.set_xlabel("Time  t"); ax1.set_ylabel("⟨X₀(t)⟩")
ax1.set_title("Trotterization: ⟨X₀⟩ — Exact vs. Trotter")
ax1.legend(fontsize=7.5); ax1.grid(True, alpha=0.3)

ax2.set_xlabel("Time  t"); ax2.set_ylabel("⟨Z₀(t)⟩")
ax2.set_title("Trotterization: ⟨Z₀⟩ — Exact vs. Trotter")
ax2.legend(fontsize=7.5); ax2.grid(True, alpha=0.3)

plt.suptitle(f"H = J·ZZ + h·X₀   J={J}, h={h},  T={T}", fontsize=12)
plt.tight_layout(); plt.show()

# Convergence table
print(f"\n{'n steps':>10}  {'Δt':>8}  {'|error| in ⟨X₀⟩':>18}")
print("-" * 42)
for n, err in zip(n_steps_list, errors):
    print(f"{n:>10}  {T/n:>8.3f}  {err:>18.6f}")
print("\nExpect error ∝ Δt (first-order Trotter) →",
      "halving Δt should ~halve error.")


## Summary

You've now simulated three canonical quantum systems and seen how **dynamics,
analytics, and circuits all describe the same physics** from different angles.

| System | H | Signature | Circuit |
|--------|---|-----------|---------|
| Larmor | ω/2 · Z | XY-plane precession, ⟨Z⟩=0 | `Rz(ωt)` |
| Rabi | Ω/2 · X | ⟨Z⟩ = cos(Ωt), π-pulse inversion | `Rx(Ωt)` |
| ZZ coupling | J · Z⊗Z | ⟨X₀⟩ = cos(2Jt) | CNOT·Rz·CNOT |
| Ising+field | J·ZZ+h·X | Trotter approximation | repeat block |

### Key Takeaways

1. **`cudaq.evolve`** solves the Schrödinger / Lindblad ODE continuously —
   no discretisation error, ideal for verifying circuit approximations.

2. **Exact circuit decompositions** exist for single-Pauli Hamiltonians
   (`Rx`, `Ry`, `Rz`, CNOT·Rz·CNOT). Use these when possible.

3. **Trotterization** handles multi-term Hamiltonians at the cost of O(Δt) error.
   Second-order Suzuki reduces this to O(Δt²) at 2× gate overhead.

4. **The dynamics → circuit → hardware pipeline** is the core of variational
   quantum simulation — CUDA-Q handles all three layers.

### Next Experiments

- Add **Lindblad collapse operators** to Part 1 to see T₁/T₂ decay
- Implement **second-order Suzuki–Trotter** and verify the error scaling
- Scale up to 8+ qubits (switch to `cudaq.set_target("nvidia")` for GPU speed)
- Replace `scipy.linalg.expm` with `cudaq.evolve` for large systems where
  classical exact diagonalization becomes infeasible

### Useful CUDA-Q Dynamics References

- `cudaq.evolve(H, dims, schedule, psi0, observables, collapse_operators, ...)`
- `cudaq.Schedule(t_list, parameter_names)`
- `cudaq.State.from_data(numpy_array)`
- `cudaq.set_target("dynamics")`

For the latest API, see: **https://nvidia.github.io/cuda-quantum/latest/api/dynamics/**
